# Геокодирование домов

По адресам получаем координаты домов как точек. Геокодирование нужно, чтобы визуализировать вакантность на карте. Геокодируем с помощью Nominatim, запущенного локально в Docker. Инструкция по запуску есть в Readme проекта [nominatim-docker](https://github.com/mediagis/nominatim-docker/tree/master/4.4). PBF-файл Москвы для Nominatim взял [отсюда](https://download.openstreetmap.fr/extracts/russia/central_federal_district/). Дома — из набора открытых данных «[Реестр домов по городу Москве](https://xn--80adsazqn.xn--p1aee.xn--p1ai/opendata/export/101)» ФРТ.

In [1]:
import pandas as pd
import requests
import tqdm

pd.set_option("display.max_columns", None)

In [2]:
houses = pd.read_csv("data/export-reestrmkd-77-20240701.zip", sep=";")

In [3]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34700 entries, 0 to 34699
Data columns (total 60 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               34700 non-null  int64  
 1   region_id                        34700 non-null  object 
 2   area_id                          1318 non-null   object 
 3   city_id                          2641 non-null   object 
 4   street_id                        33566 non-null  object 
 5   shortname_region                 34700 non-null  object 
 6   formalname_region                34700 non-null  object 
 7   shortname_area                   1318 non-null   object 
 8   formalname_area                  1318 non-null   object 
 9   shortname_city                   2641 non-null   object 
 10  formalname_city                  2641 non-null   object 
 11  shortname_street                 33566 non-null  object 
 12  formalname_street 

In [4]:
houses.sample(2)

,id,region_id,area_id,city_id,street_id,shortname_region,formalname_region,shortname_area,formalname_area,shortname_city,formalname_city,shortname_street,formalname_street,house_number,building,block,letter,address,houseguid,management_organization_id,built_year,exploitation_start_year,project_type,house_type,is_alarm,method_of_forming_overhaul_fund,floor_count_max,floor_count_min,entrance_count,elevators_count,energy_efficiency,quarters_count,living_quarters_count,unliving_quarters_count,area_total,area_residential,area_non_residential,area_common_property,area_land,parking_square,playground,sportsground,other_beautification,foundation_type,floor_type,wall_material,basement_area,chute_type,chute_count,electrical_type,electrical_entries_count,heating_type,hot_water_type,cold_water_type,sewerage_type,sewerage_cesspools_volume,gas_type,ventilation_type,firefighting_type,drainage_type
13595,7946640,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,NaN,NaN,58d3d1be-8935-4e6c-81cd-5f6eb38b5678,г,Москва,NaN,NaN,NaN,NaN,ул,Красных Зорь,49,NaN,NaN,NaN,"г. Москва, ул. Красных Зорь, д. 49",8201e785-1af5-4fc0-9763-6bc95080ad15,7820444.0,NaN,1968.0,индив.,Многоквартирный дом,Нет,Не заполнено,12.0,NaN,1.0,2.0,Не заполнено,84,NaN,NaN,"3660,00","3660,00","0,00",NaN,"435,30",NaN,498,498,NaN,Не заполнено,Не заполнено,Смешанные,"373,00",Не заполнено,1.0,Центральное,NaN,Центральное,Закрытая с приготовлением горячей воды на ЦТП,Центральное,Центральное,NaN,Центральное,Не заполнено,Не заполнено,Не заполнено
27508,8647449,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,NaN,NaN,fed18d75-b7f6-46ca-ab5f-3ce32ea358b0,г,Москва,NaN,NaN,NaN,NaN,ул,Советской Армии,6,NaN,NaN,NaN,"г. Москва, ул. Советской Армии, д. 6",78561dfd-c550-84da-1957-db89534dde5e,NaN,2013.0,2013.0,индивидуальный проект,Многоквартирный дом,Нет,На счете регионального оператора,14.0,12.0,2.0,8.0,Не заполнено,136,134.0,2.0,"36186,00","15109,20","623,91","15136,40","0,00","0,00",499,499,NaN,Свайный,Железобетонные,Монолитные,"5600,00",Отсутствует,0.0,Центральное,3.0,Центральное,Закрытая с приготовлением горячей воды на ЦТП,Центральное,Центральное,"0,00",Отсутствует,Приточно-вытяжная вентиляция,Автоматическая,Внутренние водостоки


In [8]:
houses.loc[houses["building"].notna()].head(2)

,id,region_id,area_id,city_id,street_id,shortname_region,formalname_region,shortname_area,formalname_area,shortname_city,formalname_city,shortname_street,formalname_street,house_number,building,block,letter,address,houseguid,management_organization_id,built_year,exploitation_start_year,project_type,house_type,is_alarm,method_of_forming_overhaul_fund,floor_count_max,floor_count_min,entrance_count,elevators_count,energy_efficiency,quarters_count,living_quarters_count,unliving_quarters_count,area_total,area_residential,area_non_residential,area_common_property,area_land,parking_square,playground,sportsground,other_beautification,foundation_type,floor_type,wall_material,basement_area,chute_type,chute_count,electrical_type,electrical_entries_count,heating_type,hot_water_type,cold_water_type,sewerage_type,sewerage_cesspools_volume,gas_type,ventilation_type,firefighting_type,drainage_type
27,8098046,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,NaN,NaN,0fd90ea9-84a9-4a1f-b67e-44a6f70c1ede,г,Москва,NaN,NaN,NaN,NaN,ул,1905 года,9,1,NaN,NaN,"г. Москва, ул. 1905 года, д. 9, стр. 1",5f0ede13-7339-4888-91e9-0a662ba0f7c1,8013388.0,1961.0,0.0,П-18/22,Многоквартирный дом,Нет,На счете регионального оператора,9.0,9.0,1.0,1.0,Не присвоен,67,64.0,3.0,"3343,00","2299,00","1044,00","1044,00","0,00","0,00",500,500,0,Ленточный,Железобетонные,Смешанные,"0,00",На лестничной клетке,1.0,Центральное,1.0,Центральное,Закрытая с приготовлением горячей воды на ЦТП,Центральное,Центральное,"0,00",Центральное,Приточно-вытяжная вентиляция,Отсутствует,Внутренние водостоки
28,8098048,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,NaN,NaN,0fd90ea9-84a9-4a1f-b67e-44a6f70c1ede,г,Москва,NaN,NaN,NaN,NaN,ул,1905 года,9,2,NaN,NaN,"г. Москва, ул. 1905 года, д. 9, стр. 2",d252563a-c369-4967-a136-2066a2bed31b,8013388.0,1961.0,1961.0,II-18,Многоквартирный дом,Нет,На счете регионального оператора,9.0,9.0,1.0,1.0,Не присвоен,73,65.0,8.0,"3005,00","2317,00","688,00","339,90","0,00","0,00",500,500,0,Ленточный,Железобетонные,Монолитные,"0,00",На лестничной клетке,1.0,Центральное,1.0,Центральное,Открытая с отбором сетевой воды на горячее вод...,Центральное,Центральное,"0,00",Центральное,Приточно-вытяжная вентиляция,Отсутствует,Внутренние водостоки


In [6]:
result = []
for _, row in tqdm.tqdm(houses.iterrows()):
    row = row.fillna(value=0)
    house_id = row["id"]
    street = row["formalname_street"]
    house_number = row["house_number"]
    building = row["building"]
    block = row["block"]
    letter = row["letter"]
    city = "Москва"
    
    if not (street and house_number):
        continue
    
    if building:
        house = f"{house_number}с{building}"
    elif block:
        house = f"{house_number}к{block}"
    elif letter:
        house = f"{house_number}{letter}"
    else:
        house = str(house_number)
    
    street_search = f"{house}, {street}"
    
    resp = requests.get(
        "http://localhost:8080",
        params=dict(
            street=street_search,
            city=city
        )
    )
    
    if resp.ok and len(resp.json()) > 0:
        jdata = resp.json()
        lat = jdata[0].get("lat")
        lon = jdata[0].get("lon")
    else:
        lat, lon = None, None
    
    if lat and lon:
        try:
            lat = float(lat)
            lon = float(lon)
        except ValueError:
            lat, lon = None, None
    
    result.append((house_id, lat, lon))

34700it [25:51, 22.36it/s] 


In [7]:
geo = pd.DataFrame(result, columns=["house_id", "lat", "lon"])
geo.head(2)

,house_id,lat,lon
0,8929489,55.727066,37.560168
1,8065593,55.727066,37.560168


In [8]:
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33543 entries, 0 to 33542
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   house_id  33543 non-null  int64  
 1   lat       31204 non-null  float64
 2   lon       31204 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 786.3 KB


Статистика геокодирования: 34700 домов в наборе → 33543 с адресом хотя бы в виде «улица + дом» (кое-где нет, например, улицы — их не анализируем) → 31204 с координатами (около 90%).

In [9]:
geo.to_csv("data/houses_coords.csv", index=False)